In [3]:
import numpy as np
import pandas as pd
import os
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer
import kagglehub

In [6]:
df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

In [7]:
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [8]:
print("총 샘플의 개수:",len(df))

총 샘플의 개수: 1244184


In [ ]:
# 문장데이터를 임베딩 해보자
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print("임베딩 벡터 수: ", len(encoded_data))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\user06\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user06\.cache\huggingface\hub\models--sentence-transformers--distilbert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

여기가 좀 특이한데 원래 faiss는 데이터 효율적 검색을 위한 페이스북에서 만든 c++라이브러리다
이용하기 위해서는 일단 인덱스를 정의하고 여기에 데이터를 추가해야 한다.
인덱스를 왜 정의해야 하는지는 알겠지. 그리고 아래는 정형화된 코드이다. 이 라이브러리를 쓰기 위해서.

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'abc_news')

검색 및 시간 측정

In [ ]:
def search(query):
    t = time.time()
    query_vector = model.encode([query])

    # 유사도가 높은 상위 5개 출력
    k = 5
    top_k = index.search(query_vector,k)

    print(f"total time:{time.time() - t}")

    return [data[_id] for _id in top_k[1].tolist()[0]]


In [ ]:
# 테스트
results = search("Underwater Forest Discovered")
print("results : ")
for result in results:
    print("\t", result)